In [43]:
import neologdn
import os
import urllib.request
from matplotlib.font_manager import FontProperties
import MeCab
from __future__ import unicode_literals
import re
import unicodedata
import sagemaker
from sagemaker import get_execution_role
import boto3
import json
import pandas as pd
import warnings
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import codecs
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#chaneg dummies to matrix
from scipy.sparse import csr_matrix,coo_matrix

import multiprocessing

from multiprocessing import Pool

import pickle

import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
import os
import tarfile
import string
import numpy as np

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas(desc="hoge progress: ")

import gensim



In [44]:
#tokenize japanase words
def wakati_by_mecab(text):
    tagger = MeCab.Tagger('-Ochasen -d /usr/lib64/mecab/dic/mecab-ipadic-neologd')
    tagger.parse('') 
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        pos = node.feature.split(",")[0]
        #if pos in ["名詞","動詞","形容詞","記号"]:   # 対象とする品詞
        if pos in ["名詞","動詞","形容詞"]:   # 対象とする品詞
            word = node.surface
            word_list.append(word)
        node = node.next
    return " ".join(word_list)

In [45]:
def remove_chars(row):
    cust_list=['\d+','■','\n','・','◇','①','②','③','④','【】','】','【','☆']
    del_list = string.ascii_letters + string.punctuation
    for i in del_list :
        row = row.str.replace(i,'')
    for i in cust_list :
        row = row.str.replace(i,'')
    return row

In [64]:
file_path="/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Sentiment_Analyzer/raw_data/tokenized_tcf.pkl"
with open(file_path, 'rb') as pickle_file:
    df_tcf = pickle.load(pickle_file)
    
file_path="/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Sentiment_Analyzer/raw_data/tokenized_tfc_claim.pkl"
with open(file_path, 'rb') as pickle_file:
    df_tcf_claim = pickle.load(pickle_file)

file_path="/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Sentiment_Analyzer/raw_data/tokenized_tfc_claim_test.pkl"
with open(file_path, 'rb') as pickle_file:
    test_df = pickle.load(pickle_file)

df_mutilabel=df_tcf.append(df_tcf_claim)
df_mutilabel['comment'].drop_duplicates(keep="first", inplace=True)

In [65]:
df_mutilabel['label'].value_counts()

0    32685
1     5377
Name: label, dtype: int64

In [57]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 7231 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 70 # max number of words in a question to use

def load_split_data():
    
    file_path="/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Sentiment_Analyzer/raw_data/tokenized_tcf.pkl"
    with open(file_path, 'rb') as pickle_file:
        df_tcf = pickle.load(pickle_file)
        
    file_path="/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Sentiment_Analyzer/raw_data/tokenized_tfc_claim.pkl"
    with open(file_path, 'rb') as pickle_file:
        df_tcf_claim = pickle.load(pickle_file)

    file_path="/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Sentiment_Analyzer/raw_data/tokenized_tfc_claim_test.pkl"
    with open(file_path, 'rb') as pickle_file:
        test_df = pickle.load(pickle_file)
    
    df_mutilabel=df_tcf.append(df_tcf_claim)
    df_mutilabel['comment'].drop_duplicates(keep="first", inplace=True)
    
    print("Train shape : ",df_mutilabel.shape)
    print("Test shape : ",test_df.shape)
    
    #add feature
    #df_mutilabel['num_unique_words'] = df_mutilabel['comment'].progress_apply(lambda comment: len(set(w for w in comment.split())))
    
    train_df, val_df = train_test_split(df_mutilabel, test_size=0.08, random_state=2018)

    ## fill up the missing values
    train_X = train_df["comment"].fillna("_##_").values
    val_X = val_df["comment"].fillna("_##_").values
    test_X = test_df["comment"].fillna("_##_").values

    ## Tokenize the sentences
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    val_X = tokenizer.texts_to_sequences(val_X)
    test_X = tokenizer.texts_to_sequences(test_X)

    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen)
    val_X = pad_sequences(val_X, maxlen=maxlen)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    ## Get the target values
    train_y = train_df['label'].values
    val_y = val_df['label'].values  
    
    #shuffling the data
    np.random.seed(2018)
    trn_idx = np.random.permutation(len(train_X))
    val_idx = np.random.permutation(len(val_X))

    train_X = train_X[trn_idx]
    val_X = val_X[val_idx]
    train_y = train_y[trn_idx]
    val_y = val_y[val_idx]  
    
    return train_X, val_X, test_X, train_y, val_y, tokenizer.word_index
    

In [47]:
def load_fasttext(word_index):

    #配布されていたfastTextの次元数は300だったので300
    EMBEDDING_DIM = 300
    #embedding_metrix = np.zeros(top_vocab+1,EMBEDDING_DIM)
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.uniform(size=(nb_words,EMBEDDING_DIM))
    word_vectors = gensim.models.fasttext.load_facebook_model('/home/ec2-user/SageMaker/fasttext_dic/cc.ja.300.bin')
    for word, i in word_index.items():
        if i >= nb_words: continue
        embedding_vector = word_vectors.wv[word]
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [48]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [49]:
def model_cnn(embedding_matrix):
    filter_sizes = [1,2,3,5]
    num_filters = 36

    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Reshape((maxlen, embed_size, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embed_size),
                                     kernel_initializer='he_normal', activation='elu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(maxlen - filter_sizes[i] + 1, 1))(conv))

    z = Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    outp = Dense(1, activation="sigmoid")(z)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1])
    
    return model

In [50]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [51]:
def model_lstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = Attention(maxlen)(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1])
    
    return model

In [52]:
def model_gru_srk_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = Attention(maxlen)(x) # New
    x = Dense(16, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1])
    
    return model 

In [53]:
def model_lstm_du(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1])
    return model

In [54]:
def train_pred(model, epochs=2):
    for e in range(epochs):
        model.fit(train_X, train_y, batch_size=512, epochs=epochs, validation_data=(val_X, val_y),callbacks=[checkpoints, reduce_lr])
        pred_val_y = model.predict([val_X], batch_size=1024, verbose=0)

        best_thresh = 0.5
        best_score = 0.0
        for thresh in np.arange(0.1, 0.501, 0.01):
            thresh = np.round(thresh, 2)
            score = metrics.f1_score(val_y, (pred_val_y > thresh).astype(int))
            if score > best_score:
                best_thresh = thresh
                best_score = score

        print("Val F1 Score: {:.4f}".format(best_score))

    pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)
    return pred_val_y, pred_test_y, best_score

In [55]:
checkpoints = ModelCheckpoint('weights.hdf5', monitor="val_f1", mode="max", verbose=True, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_f1', factor=0.1, patience=2, verbose=1, min_lr=0.000001)

In [30]:
train_X, val_X, test_X, train_y, val_y, word_index = load_split_data()
embedding_matrix=load_fasttext(word_index)
np.shape(embedding_matrix)

Train shape :  (38062, 2)
Test shape :  (2152, 2)


(7231, 300)

In [58]:
outputs = []
pred_val_y, pred_test_y, best_score = train_pred(model_gru_srk_atten(embedding_matrix), epochs = 5)
outputs.append([pred_val_y, pred_test_y, best_score, 'gru atten srk'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 35017 samples, validate on 3045 samples
Epoch 1/1
35017/35017 [==============================] - 17s 498us/step - loss: 0.3050 - acc: 0.8765 - f1: 0.2697 - val_loss: 0.2119 - val_acc: 0.9166 - val_f1: 0.6564

Epoch 00001: val_f1 improved from -inf to 0.65644, saving model to weights.hdf5


ValueError: Classification metrics can't handle a mix of unknown and binary targets